<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective<a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<br><strong style="color: #000000;"><em>Send a Slack notification for Unused Keypairs</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Notify-unused-keypairs"><u>Notify unused keypairs</u><a class="jp-InternalAnchorLink" href="#Notify-unused-keypairs" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find unused Keypairs</a><br>2)<a href="#2" target="_self" rel="noopener"> Send message to Slack</a></p>

In [ ]:
if region == None:
    region = ''

<h3 id="Filter-unused-Keypairs"><a id="1" target="_self" rel="nofollow"></a>Filter unused Keypairs</h3>
<p>Using unSkript's Filter AWS Unused Keypairs action, we will fetch all the available keypairs and compare them to the ones that are used by the AWS instances. If a match is not found, the keypair is deduced to be unused.</p>
<blockquote>
<p>This action takes the following parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>unused_key_pairs</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Tuple,Optional
from unskript.legos.utils import CheckOutput, CheckOutputStatus
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_filter_unused_keypairs_printer(output):
    if output is None:
        return
    if isinstance(output, CheckOutput):
        print(output.json())
    else:
        pprint.pprint(output)


@beartype
def aws_filter_unused_keypairs(handle, region: str = None) -> CheckOutput:
    """aws_filter_unused_keypairs Returns an array of KeyPair.

        :type region: object
        :param region: Object containing global params for the notebook.

        :rtype: Object with status, result of unused key pairs, and error.
    """
    all_keys_dict = {}
    used_keys_dict = {}
    key_pairs_all = []
    used_key_pairs = []
    result = []
    all_regions = [region]
    if region is None or len(region)==0:
        all_regions = aws_list_all_regions(handle=handle)
    for r in all_regions:
        try:
            ec2Client = handle.client('ec2', region_name=r)
            key_pairs_all = list(map(lambda i: i['KeyName'], ec2Client.describe_key_pairs()['KeyPairs']))
            res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
            for reservation in res:
                for keypair in reservation['Instances']:
                    if 'KeyName'in keypair and keypair['KeyName'] not in used_key_pairs:
                        used_key_pairs.append(keypair['KeyName'])
            used_keys_dict["region"]=r
            used_keys_dict["key_name"]=used_key_pairs
            all_keys_dict["region"]=r
            all_keys_dict["key_name"]=key_pairs_all
            final_dict = {}
            final_list=[]
            for k,v in all_keys_dict.items():
                if v!=[]:
                    if k=="key_name":
                        for each in v:
                            if each not in used_keys_dict["key_name"]:
                                final_list.append(each)
                if len(final_list)!=0:
                    final_dict["region"]=r
                    final_dict["unused_keys"]=final_list
            if len(final_dict)!=0:
                result.append(final_dict)
        except Exception as e:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')

task.configure(outputName="unused_keypairs")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_unused_keypairs, lego_printer=aws_filter_unused_keypairs_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Unused-Keypairs">Create List of Unused Keypairs</h3>
<p>This action filters regions that have no unused keypairs and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_unused_key_pairs</code></p>
</blockquote>

In [ ]:
all_unused_key_pairs = []
if unused_keypairs[0] == False:
    if len(unused_keypairs[1])!=0:
        all_unused_key_pairs=unused_keypairs[1]
print(all_unused_key_pairs)
task.configure(outputName="all_unused_key_pairs")

<h3 id="Send-message-to-Slack"><a id="2" target="_self" rel="nofollow"></a>Send message to Slack</h3>
<p>This action sends a message containing the region and unused keypairs list to the given channel.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)

from beartype import beartype

from beartype import beartype
@beartype
def slack_post_message_printer(output):
    if output is not None:
        pprint.pprint(output)
    else:
        return


@beartype
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfuly Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found')
        elif e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid')
        elif e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in')
        elif e.response['error'] == 'is_archived':
            raise Exception('Channel has been archived')
        elif e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long')
        elif e.response['error'] == 'no_text':
            raise Exception('Message text was not provided')
        elif e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting')
        elif e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel')
        elif e.response['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace')
        elif e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided')
        elif e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied')
        elif e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied')
        elif e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user')
        elif e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked')
        elif e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message')
        elif e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later')
        elif e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable')
        elif e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message')
        elif e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue')
        elif e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated')
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}')

    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return f"Unable to send message on {channel}"


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "channel_name",
    "message": "\\"Unused Keypairs- {}\\".format(all_unused_key_pairs)"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(channel_name)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we were able to filter unused keypairs and notify that list via slack message to the given channel. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>